In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import mne
from mne.io import read_raw_ant
from mne.utils.docs import groups
import antio
import importlib
from typing import List
from pathlib import Path

import combined_analysis_bachelor.code.functions_for_pipeline as funcs
import combined_analysis_bachelor.code.movement_detection_functions as move_functions
import combined_analysis_bachelor.code.create_processed_files as process
from my_utils.get_sub_dir import get_sub_folder_dir
from combined_analysis_bachelor.code.create_processed_files import create_processed_files

## starting with splitting continous recordings ##

In [2]:
#filepath_E_source = "E:/classification_cnt/sub_class_E.cnt"
#E = read_raw_ant(filepath_E_source, preload=True)
#onsets_E = E.annotations.onset
#label_dict_E = {
#    "rest1": [onsets_E[0], onsets_E[1]],
#    "rest2": [onsets_E[3], onsets_E[4]],
#    "rest3": [onsets_E[6], onsets_E[7]],
#    "move1": [onsets_E[9], onsets_E[10]],
#    "move2": [onsets_E[12], onsets_E[13]],
#    "move3": [onsets_E[15], onsets_E[16]],
#    "suppr1": [onsets_E[-8], onsets_E[-7]],
#    "suppr2": [onsets_E[-5], onsets_E[-4]],
#    "suppr3": [onsets_E[-2], onsets_E[-1]],
#}
#
#filepath_F_source = "E:/classification_cnt/sub_class_F.cnt"
#F = read_raw_ant(filepath_F_source, preload=True)
#onsets_F = F.annotations.onset
#label_seq_F = ["rest", "rest", "rest", "move", "move", "move", "fail", "suppr", "suppr", "suppr"]
#label_dict_F = {
#    "rest1": [onsets_F[0], onsets_F[1]],
#    "rest2": [onsets_F[3], onsets_F[4]],
#    "rest3": [onsets_F[6], onsets_F[7]],
#    "move1": [onsets_F[9], onsets_F[10]],
#    "move2": [onsets_F[12], onsets_F[13]],
#    "move3": [onsets_F[15], onsets_F[16]],
#    "suppr1": [onsets_F[-8], onsets_F[-7]],
#    "suppr2": [onsets_F[-5], onsets_F[-4]],
#    "suppr3": [onsets_F[-2], onsets_F[-1]],
#}
#
#filepath_L_source = "E:/classification_cnt/sub_class_L.cnt"
#L = read_raw_ant(filepath_L_source, preload=True)
#onsets_L = L.annotations.onset
#label_seq_L = ["rest", "move", "suppr", "rest", "move", "suppr", "rest", "move", "suppr"]
#label_dict_L = {
#    "rest1": [onsets_L[0], onsets_L[1]],
#    "move1": [onsets_L[3], onsets_L[4]],
#    "suppr1": [onsets_L[6], onsets_L[7]],
#    "rest2": [onsets_L[9], onsets_L[10]],
#    "move2": [onsets_L[12], onsets_L[13]],
#    "suppr2": [onsets_L[15], onsets_L[16]],
#    "rest3": [onsets_L[-8], onsets_L[-7]],
#    "move3": [onsets_L[-5], onsets_L[-4]],
#    "suppr3": [onsets_L[-2], onsets_L[-1]],
#}
#
#filepath_P_source = "E:/classification_cnt/sub_class_P.cnt"
#P = read_raw_ant(filepath_P_source, preload=True)
#onsets_P = P.annotations.onset
#label_seq_P = ["rest", "rest", "rest", "move", "move", "move", "suppr", "suppr", "suppr"]
#label_dict_P = {
#    "rest1": [onsets_P[0], onsets_P[1]],
#    "rest2": [onsets_P[3], onsets_P[4]],
#    "rest3": [onsets_P[6], onsets_P[7]],
#    "move1": [onsets_P[9], onsets_P[10]],
#    "move2": [onsets_P[12], onsets_P[13]],
#    "move3": [onsets_P[15], onsets_P[16]],
#    "suppr1": [onsets_P[-8], onsets_P[-7]],
#    "suppr2": [onsets_P[-5], onsets_P[-4]],
#    "suppr3": [onsets_P[-2], onsets_P[-1]],
#}
#
#classification_data_path = get_sub_folder_dir(sub="classification", data_subfolder="classification_data")
#target_dir = Path(f"{classification_data_path}/class_raw")
#
#channels_class = ["BIP3", "BIP4", "BIP5", "BIP7", "BIP8", "BIP9", "BIP10"]
#EMG = ["BIP7", "BIP8", "BIP9", "BIP10"]
#EMG_names = ["brachioradialis_L", "deltoideus_L"]
#ACC = ["acc_x_hand_L", "acc_y_hand_L", "acc_z_hand_L"]
#LOCS_left = {"BIP7":"brachioradialis_L",
#            "BIP8": "deltoideus_L",
#            "BIP3" : "acc_x_hand_L",
#            "BIP4" : "acc_y_hand_L",
#            "BIP5" : "acc_z_hand_L"}
#LOCS_right = {"BIP3" : "acc_x_hand_L",
#            "BIP4" : "acc_y_hand_L",
#            "BIP5" : "acc_z_hand_L",
#            "BIP9" : "brachioradialis_R",
#            "BIP10" : "deltoideus_R"}

In [4]:
#move_functions.split_recording_by_manual_segments(
#    filepath=filepath_P_source,
#    output_dir=target_dir,
#    segments_dict=label_dict_P,
#    channels= channels_class,
#    location_dict=LOCS_left,
#)

### we have just splitted the .cnt files and converted them into h5 ###
### Now let's process the files (notch and bandpass filter)! ###

In [9]:
classification_data_path = get_sub_folder_dir(sub="classification", data_subfolder="classification_data")
source_dir = Path(f"{classification_data_path}/class_raw")
target_processed_dir = Path(f"{classification_data_path}/class_processed")
EMG_names = ["brachioradialis_L", "deltoideus_L"]

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [38]:
importlib.reload(process)

<module 'combined_analysis_bachelor.code.create_processed_files' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\create_processed_files.py'>

In [40]:
#process.create_processed_files(source_dir, target_processed_dir, [2,20], [20,450], sf=1000)

#### and lets look at the data ####

In [41]:
check_new_filtered = f"{target_processed_dir}/sub_class_P_move1_processed.h5"

In [3]:
#comparing forearm EMG values for two subs # 
check_P_rest_proc = f"{target_processed_dir}/sub_class_P_rest1_processed.h5"
check_rest_proc_P_df = pd.read_hdf(check_P_rest_proc, key="data")

check_F_rest_proc = f"{target_processed_dir}/sub_class_F_rest1_processed.h5"
check_rest_proc_F_df = pd.read_hdf(check_F_rest_proc, key="data")

#looking at movement for sub_P
check_move = f"{target_processed_dir}/sub_class_P_move2_processed.h5"
check_move_df = pd.read_hdf(check_move, key="data")

In [4]:
check_rest_proc_P_df

,brachioradialis_L,deltoideus_L,time_sec,SVM
0,2.984279e-12,-5.060397e-13,0.000,-1.561251e-17
1,6.753690e+01,-6.283881e+00,0.001,1.864461e-03
2,1.175139e+02,1.004827e+01,0.002,3.725657e-03
3,1.089274e+02,5.754423e+00,0.003,5.576344e-03
4,2.340514e+01,6.290831e+00,0.004,7.411539e-03
...,...,...,...,...
30202,2.861575e+01,-5.981973e+01,30.202,2.946091e-02
30203,1.149216e+01,-4.690266e+01,30.203,2.230545e-02
30204,-1.898056e+01,-3.779000e+01,30.204,1.497104e-02
30205,-2.921936e+01,-1.053778e+01,30.205,7.516269e-03


In [8]:
# comparing filtered data from different subs
fig, axs = plt.subplots(2, 1)

axs[0].plot(check_rest_proc_P_df["time_sec"], check_rest_proc_P_df["SVM"])
axs[0].set_title("sub_P move")
axs[0].set_ylabel("Amplitude (microV)")

axs[1].plot(check_rest_proc_F_df["time_sec"], check_rest_proc_F_df["brachioradialis_R"])
axs[1].set_title("sub_F rest (brachioradialis)")
axs[1].set_ylabel("Amplitude (microV)")

plt.tight_layout()
plt.show()

In [4]:
# data has bigger vals at start and end --> check out how data looks with cropped edges
cropped_rest_P = funcs.crop_edges(check_rest_proc_P_df)
cropped_rest_F = funcs.crop_edges(check_rest_proc_F_df)
check_move_P = funcs.crop_edges(check_move_df)

In [53]:
# plotting cropped data
fig, axs = plt.subplots(2, 1)

axs[0].plot(cropped_rest_P["time_sec"], cropped_rest_P["brachioradialis_L"])
axs[0].set_title("sub_P rest (brachioradialis)")
axs[0].set_ylabel("Amplitude (microV)")

axs[1].plot(cropped_rest_F["time_sec"], cropped_rest_F["brachioradialis_R"])
axs[1].set_title("sub_F rest (brachioradialis)")
axs[1].set_ylabel("Amplitude (microV)")

plt.tight_layout()
plt.show()

In [45]:
importlib.reload(funcs)

<module 'combined_analysis_bachelor.code.functions_for_pipeline' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\functions_for_pipeline.py'>

# creating one big training df #

In [3]:
import combined_analysis_bachelor.code.read_in_emg_acc as read_in

In [4]:
importlib.reload(read_in)

<module 'combined_analysis_bachelor.code.read_in_emg_acc' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\read_in_emg_acc.py'>

In [5]:
# getting all of the filepaths from the folder:
from my_utils.find_paths import get_onedrive_path
SUB = "classification"

directory = get_onedrive_path(folder="classification_data")
class_filepaths, _ = read_in.get_filepaths_filenames(SUB, directory)
print(directory)

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [15]:
class_filepaths

['C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\class_processed\\external_val_data_proc',
 'C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\class_processed\\external_val_data_proc2',
 'C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\class_processed\\noisy_external_val_data_proc',
 'C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\class_processed\\old_external_val_data_proc',
 'C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\class_proce

In [6]:
df_dir = f"{directory}/sub-classification/EMG_ACC/training_dfs/training_df_full_new.h5"

In [7]:
importlib.reload(move_functions)
importlib.reload(funcs)

<module 'combined_analysis_bachelor.code.functions_for_pipeline' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\functions_for_pipeline.py'>

In [74]:
window_size = 250
overlap = 0.5
SF = 1000

move_functions.create_train_df(class_filepaths, window_size, overlap, SF, df_dir)

In [8]:
df_train = pd.read_hdf(df_dir, key="data") 

In [76]:
df_train

,sub_ID,label,window_range_ms,brachioradialis_rms,brachioradialis_mav,brachioradialis_std,brachioradialis_max,brachioradialis_zeroCrossings,brachioradialis_slopeSignChanges,brachioradialis_waveformLength,...,SVM_mean,SVM_std,SVM_max,SVM_min,SVM_range,SVM_coefVar,SVM_psd_peakfreq,SVM_psd_power_2_5Hz,SVM_psd_power_10_20Hz,SVM_psd_power_ratio_high_low
0,sub-E,move,0-250ms,75.338134,59.709425,75.333267,186.236991,34,67,5553.446854,...,0.176395,0.043762,0.245686,0.086485,0.159201,0.248093,4.0,2.194245e-04,2.825425e-04,1.281811
1,sub-E,move,125-375ms,57.152058,44.212159,57.139249,187.257614,38,76,4759.313340,...,0.142297,0.028424,0.188316,0.086485,0.101831,0.199749,8.0,6.192279e-05,5.254382e-05,0.835052
2,sub-E,move,250-500ms,103.082374,85.146622,103.081974,227.489370,29,59,7465.702160,...,0.159050,0.050481,0.268989,0.096347,0.172642,0.317389,12.0,5.498790e-05,1.596246e-04,2.851055
3,sub-E,move,375-625ms,95.798838,76.723792,95.724280,227.489370,31,51,6785.175357,...,0.179881,0.051278,0.268989,0.096347,0.172642,0.285066,4.0,4.929785e-04,5.364505e-06,0.010860
4,sub-E,move,500-750ms,52.407405,44.543869,52.407400,129.990476,33,52,4695.069323,...,0.202616,0.058109,0.294246,0.108215,0.186031,0.286794,4.0,5.682861e-04,3.810544e-05,0.066935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,sub-P,suppr,27875-28125ms,119.670602,85.656921,119.668407,340.138893,40,96,11419.496634,...,0.010987,0.004701,0.024372,0.002318,0.022054,0.427900,4.0,1.573427e-06,4.673738e-07,0.181615
224,sub-P,suppr,28000-28250ms,132.705308,97.883942,132.705239,340.138893,39,88,13608.523160,...,0.011291,0.004498,0.024372,0.005575,0.018797,0.398400,8.0,2.280884e-06,2.848773e-06,0.868294
225,sub-P,suppr,28125-28375ms,132.051305,101.137614,132.030767,466.308132,38,88,13454.912857,...,0.012120,0.004095,0.023796,0.005575,0.018221,0.337864,8.0,4.128399e-07,1.114743e-06,0.789009
226,sub-P,suppr,28250-28500ms,128.631399,98.768387,128.602888,466.308132,43,90,12692.053215,...,0.011638,0.005233,0.023796,0.002073,0.021723,0.449680,8.0,2.533818e-06,4.038012e-06,1.142677


In [11]:
df_train.keys()

Index(['sub_ID', 'label', 'window_range_ms', 'brachioradialis_rms',
       'brachioradialis_mav', 'brachioradialis_std', 'brachioradialis_max',
       'brachioradialis_zeroCrossings', 'brachioradialis_slopeSignChanges',
       'brachioradialis_waveformLength', 'deltoideus_rms', 'deltoideus_mav',
       'deltoideus_std', 'deltoideus_max', 'deltoideus_zeroCrossings',
       'deltoideus_slopeSignChanges', 'deltoideus_waveformLength',
       'brachioradialis_tkeo_rms_rms', 'brachioradialis_tkeo_rms_mav',
       'brachioradialis_tkeo_rms_std', 'brachioradialis_tkeo_rms_max',
       'brachioradialis_envelope_rms', 'brachioradialis_envelope_mav',
       'brachioradialis_envelope_std', 'brachioradialis_envelope_max',
       'brachioradialis_envelope_coefVar', 'deltoideus_tkeo_rms_rms',
       'deltoideus_tkeo_rms_mav', 'deltoideus_tkeo_rms_std',
       'deltoideus_tkeo_rms_max', 'deltoideus_envelope_rms',
       'deltoideus_envelope_mav', 'deltoideus_envelope_std',
       'deltoideus_envelope_

In [9]:
feature_cols = [i for i in df_train.columns if i not in ["label", "sub_ID", "window_range_ms"]]

In [10]:
len(feature_cols)

43

#### next step: splitting train_df according to Leave-One-Subject-Out and scaling the data ####

In [79]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [80]:
loso_data_old = move_functions.prepare_loso_data(df_train, scaling_style="sub-wise") # nochmal laufen lassen!! erst nachdem scaling angeppasst ist # vllt loso funktion ersmtal doppeln und checken was das für einen unterschied macht mit scaling

In [81]:
# check if that worked
for fold in loso_data_old:
    print("Testsubject:", fold["test_sub_ID"])
    print("Trainshape:", fold["X_train"].shape)
    print("Testshape:", fold["X_test"].shape)

Testsubject: sub-E
Trainshape: (6250, 43)
Testshape: (2043, 43)
Testsubject: sub-F
Trainshape: (6173, 43)
Testshape: (2120, 43)
Testsubject: sub-L
Trainshape: (6231, 43)
Testshape: (2062, 43)
Testsubject: sub-P
Trainshape: (6225, 43)
Testshape: (2068, 43)


#### feature selection by feature permutation ####

In [82]:
# neu laufen lassen und features vergleichen!! ist psd_ratio immer noch dabei? Oder waren alte features besser? Falls ja, alte vals verwenden, die haben dann ja anscheinend besser classes getrennt.


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# === PARAMETERS ===
IMPORTANCE_THRESHOLD = 0.01
HIGH_ACCURACY_THRESHOLD = 0.95
models = [LogisticRegression(solver='lbfgs', penalty='l2'), KNeighborsClassifier(), RandomForestClassifier()]
model_names = ["Logistic Regression", "KNN", "Random Forest"]

# === INITIALIZE ===
all_feature_scores = pd.Series(0, index=feature_cols)
feature_cols = [i for i in df_train.columns if i not in ["label", "sub_ID", "window_range_ms"]]
fold_accuracies = []
fold_feature_lists = []
all_importances_across_models = []

for i, model in enumerate(models):
    # === loop through each fold ===
    for fold_idx, fold in enumerate(loso_data_old):
        print(f"\n--- Fold {fold_idx+1}/{len(loso_data_old)} ---")
        
        X_train = pd.DataFrame(fold['X_train'], columns=feature_cols)
        X_val = pd.DataFrame(fold['X_test'], columns=feature_cols)
        y_train = fold['y_train']
        y_val = fold['y_test']
    
        # training the model
        model.fit(X_train, y_train)
    
        # Accuracy
        acc = accuracy_score(y_val, model.predict(X_val))
        fold_accuracies.append(acc)
        print(f"Accuracy: {acc:.3f}")
    
        # Permutation Importance
        importances = move_functions.permutation_importance(model, X_val, y_val)
        all_importances_across_models.append(importances)
        important_feature_indices = importances[importances > IMPORTANCE_THRESHOLD].index.tolist()
        fold_feature_lists.append(important_feature_indices)
    
        print(f"important features in fold {fold_idx+1}: {important_feature_indices}")
    
    # === feature selection strategy ===
    final_selected_indices = set()
    for acc, features in zip(fold_accuracies, fold_feature_lists):
        if acc >= HIGH_ACCURACY_THRESHOLD or len(features) > 0:
            final_selected_indices.update(features)
    
    # === index to feat name ===
    final_selected_features = list(final_selected_indices)
    
    print(f"\n=====> Final Features for {model_names[i]} over all Folds (combined):")
    print(final_selected_features)
    
    
# Combine all importance Series into a DataFrame
importance_df = pd.DataFrame(all_importances_across_models)

# Mittelwert über alle Modelle & Folds
mean_importance = importance_df.mean().sort_values(ascending=False)

# Top 5 Features
top5_features = mean_importance.head(5)

# Ausgabe + Vorbereitung für Plot
print("\n===== Top 5 features for all models & folds =====")
print(top5_features)

# Optional für späteren Plot
top5_dict = top5_features.to_dict()


--- Fold 1/4 ---
Accuracy: 0.997
important features in fold 1: ['brachioradialis_max', 'brachioradialis_waveformLength', 'deltoideus_waveformLength', 'brachioradialis_envelope_std', 'brachioradialis_envelope_max', 'SVM_min']

--- Fold 2/4 ---
Accuracy: 0.992
important features in fold 2: ['brachioradialis_rms', 'brachioradialis_mav', 'brachioradialis_std', 'brachioradialis_max', 'brachioradialis_slopeSignChanges', 'brachioradialis_waveformLength', 'deltoideus_mav', 'deltoideus_waveformLength', 'brachioradialis_tkeo_rms_rms', 'brachioradialis_tkeo_rms_mav', 'brachioradialis_tkeo_rms_max', 'brachioradialis_envelope_rms', 'brachioradialis_envelope_mav', 'brachioradialis_envelope_std', 'brachioradialis_envelope_max', 'deltoideus_envelope_mav', 'SVM_std', 'SVM_max', 'SVM_min', 'SVM_range']

--- Fold 3/4 ---
Accuracy: 0.891
important features in fold 3: ['brachioradialis_waveformLength', 'brachioradialis_envelope_rms', 'brachioradialis_envelope_mav', 'brachioradialis_envelope_max']

--- Fol

In [18]:
final_features_old = ['SVM_smooth_rms_max', 'SVM_smooth_rms_mean', 'SVM_smooth_rms_min', 'SVM_smooth_rms_psd_power_ratio_high_low', 'SVM_smooth_rms_range', 'SVM_smooth_rms_std', 'brachioradialis_envelope_coefVar', 'brachioradialis_envelope_max', 'brachioradialis_envelope_mean', 'brachioradialis_envelope_rms', 'brachioradialis_max', 'brachioradialis_rms', 'brachioradialis_slopeSignChanges', 'brachioradialis_std', 'brachioradialis_tkeo_max', 'brachioradialis_tkeo_mean', 'brachioradialis_tkeo_rms', 'brachioradialis_tkeo_slopeSignChanges', 'brachioradialis_tkeo_std', 'brachioradialis_tkeo_waveformLength', 'brachioradialis_waveformLength', 'deltoideus_envelope_coefVar', 'deltoideus_envelope_max', 'deltoideus_max', 'deltoideus_rms', 'deltoideus_slopeSignChanges', 'deltoideus_std', 'deltoideus_tkeo_mean', 'deltoideus_tkeo_rms', 'deltoideus_tkeo_waveformLength', 'deltoideus_waveformLength'] # power ratio war da noch: 1-5, 10-20

drop_feats_old = [i for i in df_train.columns if i not in final_features_old and i not in ["label", "sub_ID", "window_range_ms"]]

print(len(df_train.columns))
print(len(final_features_old))
print(len(drop_feats_old))
print(drop_feats_old)

NameError: name 'final_features' is not defined

In [19]:
# features after setting seed etc # 

final_feats_new = ['deltoideus_envelope_mav', 'brachioradialis_tkeo_rms_std', 'brachioradialis_tkeo_rms_mav', 'SVM_range', 'brachioradialis_waveformLength', 'SVM_mean', 'brachioradialis_envelope_std', 'brachioradialis_envelope_rms', 'brachioradialis_rms', 'deltoideus_waveformLength', 'brachioradialis_max', 'brachioradialis_tkeo_rms_max', 'brachioradialis_envelope_coefVar', 'SVM_psd_peakfreq', 'deltoideus_envelope_rms', 'brachioradialis_std', 'brachioradialis_envelope_max', 'deltoideus_slopeSignChanges', 'SVM_rms', 'SVM_max', 'brachioradialis_envelope_mav', 'deltoideus_mav', 'SVM_min', 'brachioradialis_mav', 'SVM_std', 'brachioradialis_slopeSignChanges', 'brachioradialis_tkeo_rms_rms']

drop_feats = [i for i in df_train.columns if i not in final_feats_new and i not in ["label", "sub_ID", "window_range_ms"]]

print(len(final_feats_new))
print(len(drop_feats))

print(drop_feats)

27
16
['brachioradialis_zeroCrossings', 'deltoideus_rms', 'deltoideus_std', 'deltoideus_max', 'deltoideus_zeroCrossings', 'deltoideus_tkeo_rms_rms', 'deltoideus_tkeo_rms_mav', 'deltoideus_tkeo_rms_std', 'deltoideus_tkeo_rms_max', 'deltoideus_envelope_std', 'deltoideus_envelope_max', 'deltoideus_envelope_coefVar', 'SVM_coefVar', 'SVM_psd_power_2_5Hz', 'SVM_psd_power_10_20Hz', 'SVM_psd_power_ratio_high_low']


In [13]:
df_picked_feats_dir = f"{directory}/sub-classification/EMG_ACC/training_dfs/training_df_final_feats_new.h5"

#df_train_final_feats = df_train.drop(drop_feats, axis=1)
#df_train_final_feats.to_hdf(df_picked_feats_dir, key="data")

In [14]:
df_train_final_feats = pd.DataFrame(pd.read_hdf(df_picked_feats_dir))

In [33]:
df_train_final_feats

,sub_ID,label,window_range_ms,brachioradialis_rms,brachioradialis_mav,brachioradialis_std,brachioradialis_max,brachioradialis_slopeSignChanges,brachioradialis_waveformLength,deltoideus_mav,...,brachioradialis_envelope_coefVar,deltoideus_envelope_rms,deltoideus_envelope_mav,SVM_rms,SVM_mean,SVM_std,SVM_max,SVM_min,SVM_range,SVM_psd_peakfreq
0,sub-E,move,0-250ms,75.338134,59.709425,75.333267,186.236991,67,5553.446854,27.320371,...,0.564097,33.453472,30.991530,0.181742,0.176395,0.043762,0.245686,0.086485,0.159201,4.0
1,sub-E,move,125-375ms,57.152058,44.212159,57.139249,187.257614,76,4759.313340,23.738568,...,0.451561,21.914914,21.900788,0.145108,0.142297,0.028424,0.188316,0.086485,0.101831,8.0
2,sub-E,move,250-500ms,103.082374,85.146622,103.081974,227.489370,59,7465.702160,22.192188,...,0.251162,22.710357,22.644147,0.166869,0.159050,0.050481,0.268989,0.096347,0.172642,12.0
3,sub-E,move,375-625ms,95.798838,76.723792,95.724280,227.489370,51,6785.175357,32.954284,...,0.294487,33.369006,31.819983,0.187047,0.179881,0.051278,0.268989,0.096347,0.172642,4.0
4,sub-E,move,500-750ms,52.407405,44.543869,52.407400,129.990476,52,4695.069323,43.607638,...,0.222750,44.130820,43.760168,0.210784,0.202616,0.058109,0.294246,0.108215,0.186031,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,sub-P,suppr,27875-28125ms,119.670602,85.656921,119.668407,340.138893,96,11419.496634,12.353044,...,0.008517,12.294989,12.127213,0.011950,0.010987,0.004701,0.024372,0.002318,0.022054,4.0
224,sub-P,suppr,28000-28250ms,132.705308,97.883942,132.705239,340.138893,88,13608.523160,13.599708,...,0.048631,13.626094,13.600069,0.012155,0.011291,0.004498,0.024372,0.005575,0.018797,8.0
225,sub-P,suppr,28125-28375ms,132.051305,101.137614,132.030767,466.308132,88,13454.912857,11.733180,...,0.038302,12.289808,12.232482,0.012793,0.012120,0.004095,0.023796,0.005575,0.018221,8.0
226,sub-P,suppr,28250-28500ms,128.631399,98.768387,128.602888,466.308132,90,12692.053215,11.190156,...,0.085919,11.179292,11.178046,0.012760,0.011638,0.005233,0.023796,0.002073,0.021723,8.0


In [15]:
loso_data = move_functions.prepare_loso_data(df_train_final_feats, scaling_style="sub-wise")

In [16]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [47]:
## plotting class and sub distribution for thesis ##

import matplotlib.pyplot as plt
from matplotlib.patches import Patch

def plot_class_balance_side_by_side(
    df,
    y_col="y_label",
    sub_col="subject",
    class_order=None,            # z.B. ["rest", "movement", "suppression"]
    class_colors=None,           # z.B. {"rest":"lightsteelblue","movement":"darkseagreen","suppression":"orchid"}
    figsize=(12, 5.0),
    save_path=None,
    dpi=600
):
    # --- Daten vorbereiten ---
    data = df.dropna(subset=[y_col]).copy()

    # Klassenreihenfolge & Farben bestimmen
    if class_order is None:
        class_order = list(data[y_col].value_counts().index)
    if class_colors is None:
        # Fallback-Farben konsistent
        default_cols = ["lightsteelblue", "darkseagreen", "orchid", "#e17c72", "#7aa6c2"]
        class_colors = {c: default_cols[i % len(default_cols)] for i, c in enumerate(class_order)}

    # Gesamtverteilung
    counts = data[y_col].value_counts().reindex(class_order, fill_value=0)
    pct = 100 * counts / counts.sum()

    # Pro-Subjekt-Proportionen (%)
    prop = (data.groupby(sub_col)[y_col]
                .value_counts(normalize=True)
                .mul(100)
                .rename("pct")
                .reset_index())
    wide = (prop.pivot(index=sub_col, columns=y_col, values="pct")
                .reindex(columns=class_order)
                .fillna(0.0))
    subs = list(wide.index)
    x = np.arange(len(subs))

    # --- Figure-Layout: 2 Panels (links Donut, rechts Bars) + kleine Legenden-Spalte ---
    fig = plt.figure(figsize=figsize, constrained_layout=False)
    gs = fig.add_gridspec(nrows=1, ncols=3, width_ratios=[1.05, 1.45, 0.55], wspace=0.25)

    ax_pie = fig.add_subplot(gs[0, 0])
    ax_bar = fig.add_subplot(gs[0, 1])
    ax_leg = fig.add_subplot(gs[0, 2])
    ax_leg.axis("off")

    # --- Donut ---
    wedges, _ = ax_pie.pie(
        counts.values,
        startangle=90,
        colors=[class_colors[c] for c in class_order],
        wedgeprops=dict(width=0.38, edgecolor="none")
    )
    ax_pie.set_aspect("equal")
    ax_pie.set_title("Overall class distribution", pad=8)

    # optional: Gesamt-N in die Mitte
    ax_pie.text(0, 0, f"N={int(counts.sum())}", ha="center", va="center", fontsize=9)

    # --- Gestapelte Balken (Pro Subjekt) ---
    bottom = np.zeros(len(subs))
    bar_w = 0.6
    for cls in class_order:
        vals = wide[cls].to_numpy()
        ax_bar.bar(x, vals, width=bar_w, bottom=bottom, color=class_colors[cls], alpha=0.7, label=str(cls))
        bottom += vals

    ax_bar.set_xticks(x)
    ax_bar.set_xticklabels(subs)
    ax_bar.set_ylim(0, 100)
    ax_bar.set_ylabel("Class share (%)")
    ax_bar.set_title("Per-subject class proportions", pad=8)
    ax_bar.set_facecolor("white")
    ax_bar.spines["top"].set_visible(False)
    ax_bar.spines["right"].set_visible(False)

    # --- Sammel-Legende rechts ---
    legend_handles = [
        Patch(facecolor=class_colors[c], edgecolor="none",
              label=f"{c} — {int(counts[c])}  ({pct[c]:.1f}%)")
        for c in class_order
    ]
    ax_leg.legend(handles=legend_handles, loc="upper left", frameon=False, title="Classes")

    # Feintuning: Tick-Dicken/Spines dünn für BA
    for a in (ax_pie, ax_bar):
        a.tick_params(width=0.6)
        for sp in a.spines.values():
            sp.set_linewidth(0.6)

    if save_path:
        if save_path.lower().endswith((".pdf", ".svg")):
            fig.savefig(save_path, bbox_inches="tight", pad_inches=0.02)
        else:
            fig.savefig(save_path, dpi=dpi, bbox_inches="tight", pad_inches=0.02)
        plt.close(fig)
    else:
        plt.show()

In [39]:
from my_utils import find_paths as find_path

saving_path = find_path.get_onedrive_path(folder="figures")
saving_path = f"{saving_path}/mix_EMG_ACC_figures/class_balance.svg"

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


In [48]:
class_order = ["rest", "move", "suppr"]
class_colors = {"rest":"lightsteelblue","move":"darkseagreen","suppr":"orchid"}

plot_class_balance_side_by_side(
    df_train_final_feats, y_col="label", sub_col="sub_ID",
    class_order=class_order, class_colors=class_colors,
    figsize=(12, 5.0),
    save_path=saving_path,
    dpi=300
)

### 1. Model with cross_val: logistic regression ###

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="logistic_regression")

logistic_regression Accuracy Ø: 0.966
Accuracy for each fold: [0.998 0.995 0.89  0.982]
rest: sensitivity Ø = 0.993, specificity Ø = 0.958
move: sensitivity Ø = 0.997, specificity Ø = 0.995
suppr: sensitivity Ø = 0.909, specificity Ø = 0.996


In [89]:
move_functions.plot_loso_roc(loso_data, model_name="logistic_regression", class_names=["rest", "move", "suppr"], mode="one-vs-all")

In [49]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [19]:
import my_utils.find_paths as find_path

In [51]:
fig_path = find_path.get_onedrive_path(folder="figures")
output = os.path.join(fig_path, "mix_EMG_ACC_figures")

move_functions.plot_cm(model_name="logistic_regression", folds_data=loso_data, sub_id="sub-E", output_dir=output, save=True)

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG


### 2. Model with cross_val : KNN ###

In [23]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="KNN")

KNN Accuracy Ø: 0.982
Accuracy for each fold: [0.994 0.997 0.947 0.989]
rest: sensitivity Ø = 0.992, specificity Ø = 0.980
move: sensitivity Ø = 0.997, specificity Ø = 0.997
suppr: sensitivity Ø = 0.956, specificity Ø = 0.995


In [ ]:
# plot: KNN aufteilung, für sub mit bester klassifizierung? 

In [53]:
move_functions.plot_cm(model_name="KNN", folds_data=loso_data, sub_id="sub-P", output_dir=output, save=True)

In [73]:
# get plots where different k-vals are used:
fold_P = loso_data[2]
X_test_P = fold_P["X_test"]
X_train_P = fold_P["X_train"]
y_test_P = fold_P["y_test"]
y_train_P = fold_P["y_train"]

move_functions.plot_knn_predictions_clean(X_train_P, X_test_P, y_train_P, y_test_P, test_sub="sub-P", k=5) 

C:\Users\User\Documents\GitHub\lid_opm\combined_analysis_bachelor\code\movement_detection_functions.py:282: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(X_test_pca[i, 0], X_test_pca[i, 1],


0.4393530997304582

### 3. Model with cross_val: Random Forest ###

In [26]:
move_functions.fit_model_acc_sens_spec(loso_data, model_name="random_forest")

random_forest Accuracy Ø: 0.972
Accuracy for each fold: [0.999 0.999 0.894 0.997]
rest: sensitivity Ø = 0.996, specificity Ø = 0.961
move: sensitivity Ø = 1.000, specificity Ø = 1.000
suppr: sensitivity Ø = 0.923, specificity Ø = 0.998


In [16]:
# spannend, wenn Patrick als test_sub genommen wurde, ist die accuracy deutlich schlechter, würde man ihn raus nehmen, wäre die mean accuracy bestimmt am besten von allen models: UPDATE: ist nicht mehr so wegen scaling!

In [ ]:
# das heißt aber auch: da wo er im training mit dabei war, ist die performance trotzdem richtig gut. Macht ja aber denke ich sinn weil: seine daten (besonders forearm) waren am noisiesten --> wenn also modell nur mit daten trainert wird, die wenig noisy sind, und test sub aber eher noisy daten hat, dass wahrscheinlich rest vom test_sub oft falsch als movement eingestuft wird würde ich mal vermuten? Kann ich das mit confusion matrix zeigen?: 

In [55]:
move_functions.plot_cm(model_name="random_forest", folds_data=loso_data, sub_id="sub-F", output_dir=output, save=True)

#### NOCHMACHEN: testen was passiert wenn ohne EMG feats? ####

### jetzt einmal mit PCA die einordnungen der subs sehen ###

In [56]:
importlib.reload(move_functions)

fold = [f for f in loso_data if f["test_sub_ID"] == "sub-P"][0]
feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]]

pca = move_functions.plot_subject_feature_space(df_train, fold, feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]])

In [44]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
print("Loadings:")
print(loadings)

plt.figure(figsize=(10, 8))
sns.heatmap(loadings, annot=True, cmap='coolwarm', xticklabels=['PC1', 'PC2'], yticklabels=feature_cols)
plt.title('Feature Importance in Principal Components')
plt.tight_layout()
plt.show()

Loadings:
[[ 4.17798765e-01  8.39130642e-01]
 [-6.43531320e-03  2.79253889e-03]
 [ 4.17801027e-01  8.39150027e-01]
 [ 3.64733670e-01  8.98561767e-01]
 [-6.58472782e-01 -1.49928530e-01]
 [-5.34336329e-01 -3.03401995e-01]
 [ 2.94940074e-01  8.45376334e-01]
 [ 6.93551405e-01 -1.70212128e-01]
 [ 5.70715211e-03 -1.17076393e-02]
 [ 6.93518633e-01 -1.70222218e-01]
 [ 6.73056189e-01 -1.68303290e-01]
 [-6.28469637e-01  7.63616559e-02]
 [-7.41536692e-01  1.96258987e-02]
 [ 6.35149821e-01 -1.89682042e-01]
 [ 2.84471380e-01  9.94133896e-01]
 [ 2.75341549e-01  9.27774238e-01]
 [ 2.87212718e-01  1.06400819e+00]
 [ 2.89162081e-01  1.04669402e+00]
 [ 9.02873942e-38  0.00000000e+00]
 [-8.30902986e-02 -5.44403140e-01]
 [ 2.58847632e-01  9.99343208e-01]
 [ 4.37925903e-01  7.98098537e-01]
 [ 4.29858734e-01  8.02314213e-01]
 [ 5.23392225e-01  3.50062023e-01]
 [ 4.87442573e-01  7.55557906e-01]
 [ 5.18601505e-01 -8.86120867e-02]
 [ 4.33386869e-01 -1.69898102e-01]
 [ 4.39027275e-01 -1.71433335e-01]
 [ 4.18613

In [103]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

In [64]:
# Split-Data (from one fold)
X_train = fold["X_train"]
y_train = fold["y_train"]
X_test = fold["X_test"]
y_test = fold["y_test"]
test_sub = fold["test_sub_ID"]
groups = fold["groups"]

X_scaled = np.vstack([X_train, X_test])
y = np.hstack([y_train, y_test])


# PCA-Plot with svm
move_functions.plot_pca_with_svm(X_scaled, y, groups, test_sub="sub-P", labels=["move", "rest", "suppr"])

In [61]:
importlib.reload(move_functions)

fold = [f for f in loso_data if f["test_sub_ID"] == "sub-F"][0]
move_functions.plot_subject_feature_space(df_train, fold, feature_cols=[col for col in df_train.columns if col not in ["sub_ID", "label", "window_range_ms"]])

,n_components,2
,copy,True
,whiten,False
,svd_solver,'auto'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,None


#### PCA for all folds ####

In [62]:
move_functions.plot_loso_pca(loso_data, feature_cols, n_top_features=20)


Top 20 positive Features for PC1:
                         feature      PC1      PC2
                  deltoideus_rms 0.234221 0.092280
                  deltoideus_std 0.234218 0.092284
         deltoideus_envelope_max 0.234116 0.094629
         deltoideus_envelope_rms 0.233357 0.091184
        deltoideus_envelope_mean 0.232675 0.090254
                  deltoideus_max 0.230463 0.091956
       deltoideus_waveformLength 0.224243 0.100340
            deltoideus_tkeo_mean 0.194305 0.102658
     deltoideus_slopeSignChanges 0.194152 0.032853
             deltoideus_tkeo_rms 0.193250 0.101824
         deltoideus_envelope_std 0.192872 0.092462
  deltoideus_tkeo_waveformLength 0.192149 0.101577
             deltoideus_tkeo_max 0.188617 0.098700
             deltoideus_tkeo_std 0.187828 0.098258
deltoideus_tkeo_slopeSignChanges 0.145527 0.045997
brachioradialis_slopeSignChanges 0.144516 0.103121
             SVM_smooth_rms_mean 0.143125 0.067963
              SVM_smooth_rms_max 0.140269 0.070

In [35]:
importlib.reload(move_functions)

<module 'combined_analysis_bachelor.code.movement_detection_functions' from 'C:\\Users\\User\\Documents\\GitHub\\lid_opm\\combined_analysis_bachelor\\code\\movement_detection_functions.py'>

## processing external val data for final tests ##

#### start with creating the final train_df (all subjects included as train subs) ####

In [103]:
# 1. read in train df again: 
train_df = pd.DataFrame(pd.read_hdf(df_picked_feats_dir, key="data"))

In [104]:
train_df

,sub_ID,label,window_range_ms,brachioradialis_rms,brachioradialis_mav,brachioradialis_std,brachioradialis_max,brachioradialis_slopeSignChanges,brachioradialis_waveformLength,deltoideus_mav,...,brachioradialis_envelope_coefVar,deltoideus_envelope_rms,deltoideus_envelope_mav,SVM_rms,SVM_mean,SVM_std,SVM_max,SVM_min,SVM_range,SVM_psd_peakfreq
0,sub-E,move,0-250ms,75.338134,59.709425,75.333267,186.236991,67,5553.446854,27.320371,...,0.564097,33.453472,30.991530,0.181742,0.176395,0.043762,0.245686,0.086485,0.159201,4.0
1,sub-E,move,125-375ms,57.152058,44.212159,57.139249,187.257614,76,4759.313340,23.738568,...,0.451561,21.914914,21.900788,0.145108,0.142297,0.028424,0.188316,0.086485,0.101831,8.0
2,sub-E,move,250-500ms,103.082374,85.146622,103.081974,227.489370,59,7465.702160,22.192188,...,0.251162,22.710357,22.644147,0.166869,0.159050,0.050481,0.268989,0.096347,0.172642,12.0
3,sub-E,move,375-625ms,95.798838,76.723792,95.724280,227.489370,51,6785.175357,32.954284,...,0.294487,33.369006,31.819983,0.187047,0.179881,0.051278,0.268989,0.096347,0.172642,4.0
4,sub-E,move,500-750ms,52.407405,44.543869,52.407400,129.990476,52,4695.069323,43.607638,...,0.222750,44.130820,43.760168,0.210784,0.202616,0.058109,0.294246,0.108215,0.186031,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,sub-P,suppr,27875-28125ms,119.670602,85.656921,119.668407,340.138893,96,11419.496634,12.353044,...,0.008517,12.294989,12.127213,0.011950,0.010987,0.004701,0.024372,0.002318,0.022054,4.0
224,sub-P,suppr,28000-28250ms,132.705308,97.883942,132.705239,340.138893,88,13608.523160,13.599708,...,0.048631,13.626094,13.600069,0.012155,0.011291,0.004498,0.024372,0.005575,0.018797,8.0
225,sub-P,suppr,28125-28375ms,132.051305,101.137614,132.030767,466.308132,88,13454.912857,11.733180,...,0.038302,12.289808,12.232482,0.012793,0.012120,0.004095,0.023796,0.005575,0.018221,8.0
226,sub-P,suppr,28250-28500ms,128.631399,98.768387,128.602888,466.308132,90,12692.053215,11.190156,...,0.085919,11.179292,11.178046,0.012760,0.011638,0.005233,0.023796,0.002073,0.021723,8.0


In [106]:
train_df.drop("window_range_ms", axis=1, inplace=True)

In [107]:
train_df

,sub_ID,label,brachioradialis_rms,brachioradialis_mav,brachioradialis_std,brachioradialis_max,brachioradialis_slopeSignChanges,brachioradialis_waveformLength,deltoideus_mav,deltoideus_slopeSignChanges,...,brachioradialis_envelope_coefVar,deltoideus_envelope_rms,deltoideus_envelope_mav,SVM_rms,SVM_mean,SVM_std,SVM_max,SVM_min,SVM_range,SVM_psd_peakfreq
0,sub-E,move,75.338134,59.709425,75.333267,186.236991,67,5553.446854,27.320371,99,...,0.564097,33.453472,30.991530,0.181742,0.176395,0.043762,0.245686,0.086485,0.159201,4.0
1,sub-E,move,57.152058,44.212159,57.139249,187.257614,76,4759.313340,23.738568,94,...,0.451561,21.914914,21.900788,0.145108,0.142297,0.028424,0.188316,0.086485,0.101831,8.0
2,sub-E,move,103.082374,85.146622,103.081974,227.489370,59,7465.702160,22.192188,93,...,0.251162,22.710357,22.644147,0.166869,0.159050,0.050481,0.268989,0.096347,0.172642,12.0
3,sub-E,move,95.798838,76.723792,95.724280,227.489370,51,6785.175357,32.954284,88,...,0.294487,33.369006,31.819983,0.187047,0.179881,0.051278,0.268989,0.096347,0.172642,4.0
4,sub-E,move,52.407405,44.543869,52.407400,129.990476,52,4695.069323,43.607638,85,...,0.222750,44.130820,43.760168,0.210784,0.202616,0.058109,0.294246,0.108215,0.186031,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,sub-P,suppr,119.670602,85.656921,119.668407,340.138893,96,11419.496634,12.353044,125,...,0.008517,12.294989,12.127213,0.011950,0.010987,0.004701,0.024372,0.002318,0.022054,4.0
224,sub-P,suppr,132.705308,97.883942,132.705239,340.138893,88,13608.523160,13.599708,124,...,0.048631,13.626094,13.600069,0.012155,0.011291,0.004498,0.024372,0.005575,0.018797,8.0
225,sub-P,suppr,132.051305,101.137614,132.030767,466.308132,88,13454.912857,11.733180,126,...,0.038302,12.289808,12.232482,0.012793,0.012120,0.004095,0.023796,0.005575,0.018221,8.0
226,sub-P,suppr,128.631399,98.768387,128.602888,466.308132,90,12692.053215,11.190156,126,...,0.085919,11.179292,11.178046,0.012760,0.011638,0.005233,0.023796,0.002073,0.021723,8.0


In [1]:
train_df.keys()

NameError: name 'train_df' is not defined

In [108]:
non_feat_cols = ["sub_ID", "label"]
feat_cols = [col for col in train_df.columns if col not in non_feat_cols]
len(feat_cols)

27

In [109]:
# scaling sub-wise:

y_train = train_df["label"].to_numpy() # labels idx of train_df and df_scaled are identical!

scaled_df = move_functions.subjectwise_zscore(train_df, "sub_ID") # scaling sub-wise

df_final = scaled_df.drop(non_feat_cols, axis=1) # dropping non-feature cols

X_train = df_final[feat_cols].to_numpy() # extracting X for training the models

27


In [110]:
lr = LogisticRegression(solver='lbfgs', penalty="l2")

lr.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [111]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [113]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [117]:
class_data_dir = get_sub_folder_dir("classification", "classification_data")
save_model_to = os.path.join(class_data_dir, "models")

project folder found: C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG
C:\Users\User\OneDrive - Charité - Universitätsmedizin Berlin\TRR295 via Teams - LID_MEG - LID_MEG\data\classification_data


In [118]:
import joblib

In [119]:
joblib.dump(lr, Path(save_model_to) / "final_logreg_new.pkl")
joblib.dump(knn, Path(save_model_to) / "final_knn_new.pkl")
joblib.dump(rf, Path(save_model_to) / "final_rf_new.pkl")

['C:\\Users\\User\\OneDrive - Charité - Universitätsmedizin Berlin\\TRR295 via Teams - LID_MEG - LID_MEG\\data\\classification_data\\sub-classification\\EMG_ACC\\models\\final_rf_new.pkl']

In [120]:
meta = {"feature_cols": feat_cols,
        "target_col": "label",
        "group_col": "sub_ID",
        "drop_after_scale": non_feat_cols,
        "scaling": "subjectwise_zscore",
        "lr": {"solver": "lbfgs"},
        "knn": "default"
}
with open(Path(save_model_to) / "final_training_meta_new.json", "w") as f:
    json.dump(meta, f, indent=2)